In [113]:
import yfinance as yf
import pandas as pd
#importando dash plotly
from dash import Dash, html, dash_table, dcc, Input, Output, callback
import plotly.express as px
import matplotlib.pyplot as plt
import dash_bootstrap_components as dbc


#carregando lista de simbolos
# lista_de_simbolos = pd.read_csv('symbols_valid_meta.csv')
# display(lista_de_simbolos.head(2))
###para testes ####
lista_de_simbolos = ['MSFT34.SA', 'PETR3.SA', 'AAPL34.SA']

#definindo Periodos
PERIODOS = ['5y','2y','1y','9mo','6mo','3mo','1mo','5d','1d'] #len 9


In [114]:

#func para criar dataframe com todas as acoes nescessarias
def creat_df(acoes:list, periodo:str = '3mo'):
    df_final = pd.DataFrame()
    dados = pd.DataFrame()
    for acao in acoes:
        dados = pd.DataFrame(yf.Ticker(acao).history(period=periodo)) #carrega o historico
        dados['Titulo'] = acao
        df_final = pd.concat([df_final,dados])
    return df_final.sort_values(by='Date')

dados = creat_df(lista_de_simbolos)
dados.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Titulo
Date,,,,,,,,
2023-09-20 00:00:00-03:00,66.714966,66.714966,65.017387,65.017387,440655,0.0,0.0,MSFT34.SA
2023-09-20 00:00:00-03:00,36.071225,36.409150,35.839502,36.119499,8157200,0.0,0.0,PETR3.SA
2023-09-20 00:00:00-03:00,43.480356,43.520321,42.741028,42.771004,166511,0.0,0.0,AAPL34.SA
2023-09-21 00:00:00-03:00,65.426800,66.405409,65.057331,65.756332,108184,0.0,0.0,MSFT34.SA
2023-09-21 00:00:00-03:00,35.771916,36.235358,35.549851,35.598125,14641100,0.0,0.0,PETR3.SA


In [115]:
#desenhando desenvolvimento acao
import plotly.graph_objects as go
def fig_candlestick(dados):
    fig = px.line(data_frame=dados,x=dados.index ,y=['Open'], color='Titulo')
    return fig

# fig_candlestick(dados)

In [116]:
# yf.Ticker('MSFT34.SA').news

In [117]:
#add controle
@callback(
    Output(component_id='grafico-01', component_property='figure'),
    Input(component_id='escolha-de-acoes', component_property='value')
)
def update_graph_01(acoes_selecionadas):
    #fintrando dados
    dados_filtrados = dados.where(dados['Titulo'].isin(list(acoes_selecionadas)))
    return fig_candlestick(dados_filtrados)
    
    

# Dash app

In [118]:
#inicializar app
app = Dash(__name__)

#definindo layout app
app.layout = dbc.Container([
    dbc.Row([
    html.H1('Acompanhamento de Ações' ,style={'backgroundColor':'white','text-align': 'center'} ),
    html.Hr(),
    html.H3('Escolha a acao:',style={'backgroundColor':'white'}),
    dcc.Dropdown(id='escolha-de-acoes', options=lista_de_simbolos, multi=True, value=lista_de_simbolos),
    dcc.Graph(figure={}, id='grafico-01')
    ]),

    html.Div(className='row', children=[
        html.Div(className='six columns', children=[
            dash_table.DataTable(data=dados[['Open','Close']].to_dict('records'), page_size=11, style_table={'overflowX': 'auto'})
        ]),
        html.Div(className='four columns', children=[
            dcc.Graph(figure=fig_candlestick(dados=dados))
        ])
    ])


])


if __name__ == '__main__':
    app.run(debug=True)